최근 자연어 처리 커뮤니티에서는 트랜스포머 기반의 여러 사전 학습모델을 묶어 만든 앙상블(ensemble) 모델이

뛰어난 성능을 보여준다.

질의응답 직업 벤치마크 결과를 섞어 만든 SQuAD2.0에 최고 성능이 모델 10개가 모두 앙상블 모델일 정도입니다.

In [2]:
datasets = [ 
    ['What music do you like?', 'I like Rock music.', 1],
    ['What is your favorite food?', 'I like sushi the best', 1],
    ['What is your favorite color?', "I'm going to be a doctor", 0],
    ['What is your favorite song?', "Tokyo olympic game in 2020 was postponed", 0],
    ['Do you like watching TV shows?', "Yeah, I often watch it in my spear time", 1]
]

In [5]:
# BERT 앙상블 클래스 정의
# NSP 작업을 위해 두 개의 BERT를 병행 연결하고 마지막 층은 선형 결합층이 되도록 
# 앙상블 학습 클래스를 정의하시오. BertPreTrainedModel 클래스로부터 상속을 받아
# 사전학습(pre-training)과 재학습(re-training)이 가능합니다. 클래스 개념은 문제 002를 상속 개념은 008문제를 참조하시오.

from transformers import BertPreTrainedModel, BertConfig, BertModel, BertTokenizer, AdamW
from torch import nn
import torch

# 클래스 정의
class BertEnsembleForNextSentencePrediction(BertPreTrainedModel):

    # 생성자 설정
    def __init__(self, config, *args, **kwargs):
        super().__init__(config)

        # QA BERT 모델
        self.bert_model_1 = BertModel(config)
        # AQ BERT 모델
        self.bert_model_2 = BertModel(config)

        # Linear function
        self.cls = nn.Linear(2 * self.config.hidden_size, 2)

        # initial weight
        self.init_weights()

    # forward 신경망 설정
    def forward(
        self,
        input_ids=None,
        attention_mask=None,
        token_type_ids=None,
        position_ids=None,
        head_mask=None,
        inputs_embeds=None,
        next_sentence_label=None
    ):
        outputs = []
        
        # 첫 번째 입력 문자 저장
        input_ids_1 = input_ids[0]

        # input_ids 첫 번째 입력 문장의 attention mask 
        attention_mask_1 = attention_mask[0]

        # Bert_model_1에 input_ids1 투입한 결과를 outputs에 순차적으로 저장
        outputs.append(self.bert_model_1(input_ids_1, attention_mask=attention_mask_1))

        # input_ids 두 번째 입력(문장) 저장
        input_ids_2 = input_ids[1]

        # inputs 두 번째 입력 문장의 attention_mask 저장
        attention_mask_2 = attention_mask[1]

        # bert_model_2에 input_ids2 투입한 결과를 outputs에 순차적으로 저장
        outputs.append(self.bert_model_2(input_ids_2, attention_mask=attention_mask_2))

        # outputs에 쌓인 otuput의 두 번째 요소(output[1])를 하나씩 추출하여
        # torch.cat()으로 토치 텐서 형태로 병합
        # 이를 통해 마지막 은닉층 임베딩 상태를 구함.
        last_hidden_states = torch.cat([output[1] for output in outputs], dim=1)

        # self.cls 선형함수에 마지막 은닉층 임베딩 상태를 투입하여 로짓 추출
        logits = self.cls(last_hidden_states)

        # 크로스 엔트로피 손실(crossentropyloss) 구하기
        if next_sentence_label is not None:
            # nn.CrossEntropyLoss( ) 입력 데이터의 마지막 인덱스는 계산에서 제외
            loss_fct = nn.CrossEntropyLoss(ignore_index=-1)
            # logits.view(-1, 2)는 열이 두 개 형태로 logits를 정렬
            # next_sentence_label.view(-1)는 행이 하나인 형태로 정렬
            next_sentence_label = loss_fct(logits.view(-1, 2), next_sentence_label.view(-1))
            return next_sentence_label, logits
        else:
            return logits

In [6]:
# 앙상블 트레이닝에 사용할 사전학습 BERT 불러오기
# BERT 앙상블 학습 클래스를 인스턴스화하고 이를 GPU에 전달하세요. 아울러
# Optimizer 변수에 최적화 함수로 AmdmW를 대입하세요 가중치 감소 가능을 통해 과적합을 방지합니다.
import torch
from transformers import AdamW

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 모델 및 Config 설정
config = BertConfig()
model = BertEnsembleForNextSentencePrediction(config)

# 토크나이저 설정
model.to(device)
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')

# 학습률 설정
learning_rate = 1e-5

# 절편과 가중치를 설정.
no_decay = ["bias", "LayerNorm.weight"]

# 최적화 함수 그룹 파라미터 설정
optimizer_grouped_parameters = [{
    "params": [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
}]
# 최적화 함수 설정
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)

/mnt/d/workspace/colab_try/.venv/lib/python3.10/site-packages/transformers/optimization.py:640: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [15]:
# 060 BERT 앙상블 학습 - 데이터 증강
def prepare_data(dataset, qa=True):
    input_ids, attention_masks = [], []
    labels = []

    for point in dataset:
        if qa is True:
            # point에 있는 3개의 원소를 앞에 요소부터 q, a, _으로 배경
            q, a, _ = point
        else:
            a, q, _ = point

        encoded_dict = tokenizer.encode_plus(
            q,
            a,
            add_special_tokens=True,
            max_length=128,
            pad_to_max_length=True,
            return_attention_mask=True,
            return_tensors='pt',
            truncation=True
        )
        input_ids.append(encoded_dict["input_ids"])

        attention_masks.append(encoded_dict["attention_mask"])

        labels.append(point[-1])

    input_ids = torch.cat(input_ids, dim=0)
    
    attention_masks = torch.cat(attention_masks, dim=0)

    return input_ids, attention_masks, labels

In [16]:
# 061 BERT앙상블 학습 커스텀 데이터 세트 정의
import numpy as np
from torch.utils.data import DataLoader, RandomSampler, Dataset, SequentialSampler

# QADatasets 클래스 생성.
class QADataset(Dataset):

    # input_ids 텐서와 attention_masks 텐서 생성
    def __init__(self, input_ids, attention_masks, labels=None):
        self.input_ids = np.array(input_ids)
        self.attention_mask = np.array(attention_masks)
        # torch.long은 정수 (integer)타입을 의미
        self.labels = torch.tensor(labels, dtype=torch.long)
    
    def __getitem__(self, index):
        return self.input_ids[index], self.attention_mask[index], self.labels[index]

    def __len__(self):
        return self.input_ids.shape[0]

In [18]:
# BERT 앙상블 학습 데이타 로더
input_ids_qa, attention_masks_qa, labels_qa = prepare_data(datasets)

train_dataset_qa = QADataset(input_ids_qa, attention_masks_qa, labels_qa)

input_ids_aq, attention_masks_aq, labels_aq = prepare_data(datasets, qa=False)

train_dataset_aq = QADataset(input_ids_aq, attention_masks_aq, labels_aq)

dataloader_qa = DataLoader(
    dataset=train_dataset_qa,
    batch_size=5,
    sampler=SequentialSampler(train_dataset_qa)
)

dataloader_aq = DataLoader(
    dataset=train_dataset_aq,
    batch_size=5,
    sampler=SequentialSampler(train_dataset_aq)
)

/tmp/ipykernel_47482/2364885428.py:10: DeprecationWarning: __array__ implementation doesn't accept a copy keyword, so passing copy=False failed. __array__ must implement 'dtype' and 'copy' keyword arguments.
  self.input_ids = np.array(input_ids)
/tmp/ipykernel_47482/2364885428.py:11: DeprecationWarning: __array__ implementation doesn't accept a copy keyword, so passing copy=False failed. __array__ must implement 'dtype' and 'copy' keyword arguments.
  self.attention_mask = np.array(attention_masks)


In [24]:
# BERT 앙상블 학습 파인튜닝.
from tqdm import tqdm
epochs = 30
progress = tqdm(range(epochs))
for epoch in progress:

    for step, combine_batch in enumerate(zip(dataloader_qa, dataloader_aq)):
        batch_1, batch_2 = combine_batch
        # 모델 
        model.train()

        batch_1 = tuple(t.to(device) for t in batch_1)
        batch_2 = tuple(t.to(device) for t in batch_2)

        inputs = {
            "input_ids": [batch_1[0], batch_2[0]],
            "attention_mask": [batch_1[1], batch_2[1]],
            "next_sentence_label": batch_1[2]
        }

        outputs = model(**inputs)

        loss = outputs[0]

        loss.backward()

        # print(f"epoch: {epoch}, loss: {loss}")
        progress.set_postfix_str(f"epoch: {epoch}, loss: {loss}")
        optimizer.step()

        model.zero_grad()

100%|██████████| 30/30 [00:02<00:00, 11.19it/s, epoch: 29, loss: 0.00048139350838027894]


In [29]:
input_ids_qa, attention_masks_qa, labels_qa = prepare_data(datasets)

test_dataset_qa = QADataset(input_ids_qa, attention_masks_qa, labels_qa)

input_ids_aq, attention_masks_aq, labels_aq = prepare_data(datasets, qa=False)

test_dataset_aq = QADataset(input_ids_aq, attention_masks_aq, labels_aq)

dataloader_qa = DataLoader(
    dataset=test_dataset_qa,
    batch_size=16,
    sampler=SequentialSampler(test_dataset_qa)
)


dataloader_aq = DataLoader(
    dataset=test_dataset_aq,
    batch_size=16,
    sampler=SequentialSampler(test_dataset_aq)
)

complete_outputs, complete_label_ids = [], []

for step, combine_batch in enumerate(zip(dataloader_qa, dataloader_aq)):
    model.eval()

    batch_1, batch_2 = combine_batch

    batch_1 = tuple(t.to(device) for t in batch_1)
    batch_2 = tuple(t.to(device) for t in batch_2)

    with torch.no_grad():
        inputs = {
            "input_ids": [batch_1[0], batch_2[0]],
            "attention_mask": [batch_1[1], batch_2[1]],
            "next_sentence_label": batch_1[2]
        }

        outputs = model(**inputs)

        tmp_eval_loss, logits = outputs[:2]

        logits = logits.detach().cpu().numpy()

        outputs = np.argmax(logits, axis=1)

        labels_ids = inputs['next_sentence_label'].detach().cpu().numpy()

    complete_outputs.extend(outputs)
    complete_label_ids.extend(labels_ids)
    
print(complete_outputs, complete_label_ids)

/tmp/ipykernel_47482/2364885428.py:10: DeprecationWarning: __array__ implementation doesn't accept a copy keyword, so passing copy=False failed. __array__ must implement 'dtype' and 'copy' keyword arguments.
  self.input_ids = np.array(input_ids)
/tmp/ipykernel_47482/2364885428.py:11: DeprecationWarning: __array__ implementation doesn't accept a copy keyword, so passing copy=False failed. __array__ must implement 'dtype' and 'copy' keyword arguments.
  self.attention_mask = np.array(attention_masks)


[np.int64(1), np.int64(1), np.int64(0), np.int64(0), np.int64(1)] [np.int64(1), np.int64(1), np.int64(0), np.int64(0), np.int64(1)]


In [33]:
datasets = [["what music do you like?", "I like Rock Music", 1]]

input_ids_qa, attention_masks_qa, labels_qa = prepare_data(datasets)

test_dataset_qa = QADataset(input_ids_qa, attention_masks_qa, labels_qa)

input_ids_aq, attention_masks_aq, labels_aq = prepare_data(datasets, qa=False)

test_dataset_aq = QADataset(input_ids_aq, attention_masks_aq, labels_aq)

dataloader_qa = DataLoader(
    dataset=test_dataset_qa,
    batch_size=16,
    sampler=SequentialSampler(test_dataset_qa)
)


dataloader_aq = DataLoader(
    dataset=test_dataset_aq,
    batch_size=16,
    sampler=SequentialSampler(test_dataset_aq)
)

complete_outputs, complete_label_ids = [], []

for step, combine_batch in enumerate(zip(dataloader_qa, dataloader_aq)):
    model.eval()

    batch_1, batch_2 = combine_batch

    batch_1 = tuple(t.to(device) for t in batch_1)
    batch_2 = tuple(t.to(device) for t in batch_2)

    with torch.no_grad():
        inputs = {
            "input_ids": [batch_1[0], batch_2[0]],
            "attention_mask": [batch_1[1], batch_2[1]],
            "next_sentence_label": batch_1[2]
        }

        outputs = model(**inputs)

        tmp_eval_loss, logits = outputs[:2]

        logits = logits.detach().cpu().numpy()

        outputs = np.argmax(logits, axis=1)

        labels_ids = inputs['next_sentence_label'].detach().cpu().numpy()

    complete_outputs.extend(outputs)
    complete_label_ids.extend(labels_ids)

print(complete_outputs, complete_label_ids)

/mnt/d/workspace/colab_try/.venv/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:2690: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
/tmp/ipykernel_47482/2364885428.py:10: DeprecationWarning: __array__ implementation doesn't accept a copy keyword, so passing copy=False failed. __array__ must implement 'dtype' and 'copy' keyword arguments.
  self.input_ids = np.array(input_ids)
/tmp/ipykernel_47482/2364885428.py:11: DeprecationWarning: __array__ implementation doesn't accept a copy keyword, so passing copy=False failed. __array__ must implement 'dtype' and 'copy' keyword argume

[np.int64(1)] [np.int64(1)]
